<a href="https://colab.research.google.com/github/VenumiV/Credit-Card-Fraud-Detection-using-Machine-Learning/blob/main/Credit_Card_Fraud_Detection_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/credit_data.csv', on_bad_lines='skip', low_memory=False)


In [21]:
#first 5 rows of the dataset
credit_card_data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462388,0.239598554061257,0.0986979012610507,0.363786969611213,0.090794,-0.551599533260813,-0.617800855762348,-0.991390,-0.311169,1.46817697209427,-0.470401,0.207971,0.025791,0.403993,0.251412098239705,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0.0,1.191857,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.082361,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974,1.61272666105479,1.06523531137287,0.489095,-0.143772,0.635558093258208,0.463917,-0.114805,-0.183361,-0.145783,-0.0690831352230203,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1.0,-1.358354,-1.34016307473609,1.77320934263119,0.379779593034328,-0.503198133318193,1.800499,0.791460956450422,0.247675786588991,-1.51465432260583,0.207643,0.624501459424895,0.066083685268831,0.717293,-0.165946,2.34586494901581,-2.890083,1.109969,-0.121359,-2.261857,0.524979725224404,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1.0,-0.966272,-0.185226008082898,1.79299333957872,-0.863291275036453,-0.0103088796030823,1.247203,0.23760893977178,0.377435874652262,-1.38702406270197,-0.054952,-0.226487263835401,0.178228225877303,0.507757,-0.287924,-0.631418117709045,-1.059647,-0.684093,1.965775,-1.232622,-0.208037781160366,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2.0,-1.158233,0.877736754848451,1.548717846511,0.403033933955121,-0.407193377311653,0.095921,0.592940745385545,-0.270532677192282,0.817739308235294,0.753074,-0.822842877946363,0.53819555014995,1.345852,-1.119670,0.175121130008994,-0.451449,-0.237033,-0.038195,0.803487,0.408542360392758,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [22]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
393710,172786.0,-11.881118,10.0717849710003,-9.83478345739033,-2.06665568459413,-5.36447278097585,-2.606837,-4.91821543115252,7.305334020798,1.9144282734458,4.356170,-1.59310526246153,2.71194079105717,-0.689256,4.626942,-0.92445871482528,1.107641,1.991691,0.510632,-0.682920,1.4758291346555,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0.0
393711,172787.0,-0.732789,-0.0550804899173326,2.03502974528243,-0.73858858438744,0.868229398914682,1.058415,0.0243296959175797,0.294868698501783,0.584800017281683,-0.975926,-0.150188847103276,0.915801914403555,1.214756,-0.675143,1.16493090944588,-0.711757,-0.025693,-1.221179,-1.545556,0.0596158998872689,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0.0
393712,172788.0,1.919565,-0.301253845990644,-3.24963981406834,-0.55782812475002,2.63051512011547,3.031260,-0.296826527116156,0.708417184967134,0.432454047632915,-0.484782,0.411613736794327,0.0631188625621446,-0.183699,-0.510602,1.32928351250595,0.140716,0.313502,0.395652,-0.577252,0.00139597028995166,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0.0
393713,172788.0,-0.240440,0.530482513118839,0.702510230095103,0.689799168040973,-0.377961134444982,0.623708,-0.68617998628885,0.679145459790659,0.392086712465972,-0.399126,-1.93384881505713,-0.962886142890271,-1.042082,0.449624,1.96256312066577,-0.608577,0.509928,1.113981,2.897849,0.127433515805355,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0.0
393714,172792.0,-0.533413,-0.189733337002305,0.703337366963779,-0.506271240328258,-0.0125456787599659,-0.649617,1.57700625437629,-0.414650407552662,0.486179505267237,-0.915427,-1.04045833522361,-0.0315130540252157,-0.188093,-0.084316,0.0413334553360658,-0.302620,-0.660377,0.167430,-0.256117,0.382948104875066,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0.0


In [24]:
# dataset information
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393715 entries, 0 to 393714
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    393715 non-null  float64
 1   V1      393715 non-null  float64
 2   V2      393715 non-null  object 
 3   V3      393715 non-null  object 
 4   V4      393715 non-null  object 
 5   V5      393715 non-null  object 
 6   V6      393713 non-null  float64
 7   V7      393713 non-null  object 
 8   V8      393713 non-null  object 
 9   V9      393710 non-null  object 
 10  V10     393709 non-null  float64
 11  V11     393707 non-null  object 
 12  V12     393702 non-null  object 
 13  V13     393701 non-null  float64
 14  V14     393701 non-null  float64
 15  V15     393698 non-null  object 
 16  V16     393692 non-null  float64
 17  V17     393688 non-null  float64
 18  V18     393684 non-null  float64
 19  V19     393681 non-null  float64
 20  V20     393674 non-null  object 
 21  V21     39

In [25]:
#checking the no of missing values in each column
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,2
V7,2
V8,2
V9,5


Methods to Handle Missing Values
1.Imputation
2.Dropping

In [27]:
credit_card_data.shape

(393715, 31)

In [28]:
#dropping missing values
credit_card_data = credit_card_data.dropna()


In [30]:
#check null values are gone
credit_card_data.isnull().sum()


,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [31]:
#check the ditribution of legit transaction or legit tranactions and fraudlent transaction
credit_card_data['Class'].value_counts()

,count
Class,
0.0,392942
1.0,694


This dataset is highly unbalanced

0 ----> Normal Transactions
1 ----->Fraudlent Transactions

In [9]:
#sepereating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [34]:
print(legit.shape)
print(fraud.shape)

(392942, 31)
(694, 31)


In [35]:
#statistical measures of the data
legit.Amount.describe()

,Amount
count,392942.000000
mean,86.764243
std,246.357088
min,0.000000
25%,5.300000
50%,21.000000
75%,75.407500
max,25691.160000


In [7]:
#compare the values for both transactions
credit_card_data.groupby('Class').mean(numeric_only=True)


,Time,V1,V6,V10,V13,V14,V16,V17,V18,V19,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,
0.0,88874.771493,-0.037046,0.014155,0.002497,0.011245,0.018576,0.005120,0.016553,-0.005141,-0.006689,-0.010984,-0.023864,-0.004732,0.003444,0.023021,0.002785,125.918639,0.000276,86.764243
1.0,80031.299712,-4.839922,-1.242459,-5.820151,-0.145453,-6.748667,-4.208607,-6.790674,-2.303665,0.712464,0.703117,0.048601,0.010530,-0.093427,0.016868,0.026857,0.084424,0.069642,122.688127


Under-Sampling

Build a sample datset from the original dataset containing similar distribution of normal transaction and fraudlent transaction

Number of Fraudlent transapctions ->
(694, 31)

In [10]:
legit_sample = legit.sample(n=694)

Concatenating 2 data frames

In [11]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [12]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
200605,104470.0,-0.583787,0.717856031457844,2.32474138717518,-0.347231883132579,0.207298626334646,-0.358167,0.554266002405569,-0.23652908882282,1.64560669753371,...,-0.318678,-0.533328,-0.296048,-0.234103,0.321060,-0.777450,-0.099238,-0.150406,11.27,0.0
212900,114194.0,0.197234,-0.161626433989713,1.06290880803053,-1.74367754312734,-0.699706636780951,-0.637575,-0.0121439933094245,-0.0547936363753651,-0.664683185377247,...,0.056794,0.081772,0.205100,-0.044126,-0.744839,-0.558076,0.081017,0.056638,38.00,0.0
130574,75931.0,-0.546734,0.167045225265107,2.08515946867487,-0.207312699024184,-1.33498382913823,-0.439138,0.875064563262561,-0.22372795186027,0.786583529499911,...,0.218765,0.826127,0.101913,0.928870,-0.896121,0.218684,-0.084152,-0.056613,159.50,0.0
182057,87709.0,2.119177,-0.0212839808425694,-1.97009357034096,-0.325872068083434,0.368062989029209,-1.538390,0.614313539102652,-0.516461066437562,0.0384904347628616,...,0.077920,0.286809,0.149789,1.194315,0.038422,1.294681,-0.172815,-0.080382,9.99,0.0
122710,74432.0,0.820044,0.36153181212696,-0.876831366590174,-0.67821902971761,-0.436603751032088,-1.142447,-0.0703146463802538,0.00905034613706221,-1.56086545513783,...,0.330089,0.881637,0.134333,-0.048977,-0.433417,-0.180488,-0.172227,-0.206584,15.00,0.0


In [13]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
388771,169142.0,-1.927883,1.12565266367046,-4.51833064132991,1.74929253346589,-1.56648729197754,-2.010494,-0.882849830858081,0.697211109261327,-2.0649452874475,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1.0
389051,169347.0,1.378559,1.28938093711056,-5.00424678441137,1.4118498419441,0.442580635567782,-1.326536,-1.41316995590712,0.248525467627175,-1.12739593417081,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1.0
389057,169351.0,-0.676143,1.1263660623459,-2.21369952308058,0.46830838758824,-1.12054104443306,-0.003346,-2.23473929608742,1.21015796383769,-0.65224992035649,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1.0
390052,169966.0,-3.113832,0.58586417180689,-5.39973021073242,1.81709247345531,-0.840618465991056,-2.943548,-2.20800192003372,1.05873267723056,-1.63233334974982,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1.0
390582,170348.0,1.991976,0.158475887304227,-2.58344064503516,0.408669992998441,1.15114706077937,-0.096695,0.223050267455537,-0.0683838777747007,0.577829383844873,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1.0


In [14]:
new_dataset ['Class'].value_counts()

,count
Class,
0.0,694
1.0,694


In [16]:
new_dataset.groupby('Class').mean(numeric_only=True)


,Time,V1,V6,V10,V13,V14,V16,V17,V18,V19,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,
0.0,90983.282421,-0.046019,-0.086269,0.015740,0.003994,0.041102,-0.016245,0.016468,0.041072,-0.035909,-0.021492,-0.037495,-0.007845,0.018601,0.005006,0.003092,-0.013046,-0.004110,91.987997
1.0,80031.299712,-4.839922,-1.242459,-5.820151,-0.145453,-6.748667,-4.208607,-6.790674,-2.303665,0.712464,0.703117,0.048601,0.010530,-0.093427,0.016868,0.026857,0.084424,0.069642,122.688127


Splitting the data into features and targets

In [17]:

X = new_dataset.drop(columns='Class',axis=1)
Y = new_dataset['Class']


In [18]:
print(X)

            Time        V1                   V2                  V3  \
200605  104470.0 -0.583787    0.717856031457844    2.32474138717518   
212900  114194.0  0.197234   -0.161626433989713    1.06290880803053   
130574   75931.0 -0.546734    0.167045225265107    2.08515946867487   
182057   87709.0  2.119177  -0.0212839808425694   -1.97009357034096   
122710   74432.0  0.820044     0.36153181212696  -0.876831366590174   
...          ...       ...                  ...                 ...   
388771  169142.0 -1.927883     1.12565266367046   -4.51833064132991   
389051  169347.0  1.378559     1.28938093711056   -5.00424678441137   
389057  169351.0 -0.676143      1.1263660623459   -2.21369952308058   
390052  169966.0 -3.113832     0.58586417180689   -5.39973021073242   
390582  170348.0  1.991976    0.158475887304227   -2.58344064503516   

                        V4                  V5        V6                   V7  \
200605  -0.347231883132579   0.207298626334646 -0.358167    0.5542

Split the data into Training data & Testing Data

In [19]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [21]:
print(X.shape, X_train.shape, X_test.shape)

(1388, 30) (1110, 30) (278, 30)


Model Training

Logistic Regression

In [22]:
model = LogisticRegression()

In [23]:
# training the logistic regression model with training data
model.fit(X_train, Y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evaluation

Accuracy Score

In [24]:
#accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [25]:
print('Accuracy on Training Data: ',training_data_accuracy)

Accuracy on Training Data:  0.9405405405405406


In [26]:
#accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [27]:
print('Accuracy score on Test data: ',test_data_accuracy)

Accuracy score on Test data:  0.9424460431654677
